<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Redes-neuronales-basadas-en-ADN/blob/main/DNA_based_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
encoding={
      "A":"0",
      "T":"1",
      "C":"2",
      "G":"3",
      '0':"A",
      '1':"T",
      '2':"C",
      '3':"G"
  }

complement = {
    "A":"T",
    "T":"A",
    "G":"C",
    "C":"G"
}


SIGN = {
    "A":1,
    "T":-1,
    "G":1,
    "C":-1,
}

TOKEN_WIDTH=20


In [ ]:
def DNA_encoder(f):
  def _f(num: float, signed = True):
    sign = ""
    if signed:
      sign = 'A'
      if num < 0:
        sign = 'T'
    tmp = f(abs(num))
    return sign+"".join([encoding[i] if i!='.' else '.' for i in tmp])
  return _f

def DNA_decoder(f):
  def _f(seq: str, signed = True):
    sign = 1
    if signed:
      sign = SIGN[seq[0]]
      seq = seq[1:]
    tmp = "".join([encoding[i] if i!='.' else '.' for i in seq])
    return sign*f(tmp)
  return _f

In [ ]:
def invert(seq: str):
  return "".join([complement[i] if i!='.' else '.' for i in seq])

In [ ]:
@DNA_encoder
def float_to_base4(num :float):
    integer_part = int(num)
    fractional_part = num - integer_part

    base4_integer = ""
    if integer_part == 0:
        base4_integer = "0"
    else:
        while integer_part > 0:
            base4_integer = str(integer_part % 4) + base4_integer
            integer_part //= 4

    base4_fractional = ""
    while fractional_part > 0 and len(base4_fractional) < TOKEN_WIDTH:
        fractional_part *= 4
        digit = int(fractional_part)
        base4_fractional += str(digit)
        fractional_part -= digit
    return base4_integer.rjust(TOKEN_WIDTH,'0') + "." + (base4_fractional if base4_fractional else "").ljust(TOKEN_WIDTH,'0')

@DNA_decoder
def base4_to_float(base4):
    if "." in base4:
        integer_part, fractional_part = base4.split(".")
    else:
        integer_part, fractional_part = base4, ""

    base10_integer = 0
    for i, digit in enumerate(reversed(integer_part)):
        base10_integer += int(digit) * (4 ** i)

    base10_fractional = 0
    for i, digit in enumerate(fractional_part):
        base10_fractional += int(digit) * (4 ** -(i + 1))

    return base10_integer + base10_fractional

number = -123.0
result = float_to_base4(number)
print(result)
result1 = base4_to_float(result)
print(result1)
result1 = base4_to_float(invert(result))
print(result1)



#Red ADN

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

model = Sequential([
    Dense(4, input_dim=5, activation='linear', use_bias=True),
    Dense(7, activation='linear', use_bias=True),
    Dense(1, activation='linear', use_bias=True)
    ])

# Compilar el modelo (requerido en Keras)
model.compile(optimizer='sgd', loss='mean_squared_error')

# Ver los pesos iniciales del modelo
initial_weights = model.get_weights()
print("Pesos iniciales:")
print(f"Pesos: {initial_weights}")  # Pesos de las conexiones

# Definir nuevos pesos y sesgos
new_weights = initial_weights.copy()
# Reemplazar los pesos y el sesgo del modelo
model.set_weights(new_weights)

# Verificar que los pesos fueron actualizados
updated_weights = model.get_weights()
print("\nPesos actualizados:")
print(f"Pesos: {updated_weights}")

In [ ]:
for i in range(0,len(initial_weights),2):
  print(initial_weights[i],initial_weights[i+1],end='\n\n')

In [ ]:
def recursiveEncoder(weights):
  if not (isinstance(weights, list) or isinstance(weights, np.ndarray)):
    return float_to_base4(weights)
  out = []
  for i in weights:
    out.append(recursiveEncoder(i))
  return out


def recursiveDecoder(weights):
  if not (isinstance(weights, list) or isinstance(weights, np.ndarray)):
    return base4_to_float(weights)
  out = []
  for i in weights:
    out.append(recursiveDecoder(i))
  try:
    return np.array(out)
  except:
    return out

print(
    recursiveEncoder(33123.543534),
    tmp:=recursiveEncoder(initial_weights),
    recursiveDecoder(tmp),
    sep = '\n\n'
)

In [ ]:
model.set_weights(recursiveDecoder(tmp))

## Algoritmo genético

### preparación de los datos

In [ ]:
import seaborn as sns
import pandas as pd
data = sns.load_dataset('iris')
X = data[['sepal_length', 'sepal_width', 'petal_length','petal_width']]
Y = pd.get_dummies(data['species'])

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TerminateOnNaN
from tensorflow import keras

In [ ]:
layers = [
    Dense(4, activation='relu', use_bias=True),
    Dense(30, activation='sigmoid', use_bias=True),
    Dense(70, activation='sigmoid', use_bias=True),
    Dense(70, activation='sigmoid', use_bias=True),
    Dense(3, activation='softmax')
    ]

In [ ]:
initial_sols = 5

sols = []
error = []

limit = 5
epoch = 10

# probabilidad de invertirse
Pi=.2
# Probabilidad de mutar
pm=.5
# Probabilidad de cruce
pc=.3

In [ ]:
for i in range(initial_sols):
  print("Prueba")
  model = Sequential(layers)
  model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

  hist = model.fit(X,
      Y,
      batch_size=96,
      epochs=2000,
      validation_split=.3,
      callbacks=[
      EarlyStopping(monitor="val_loss",restore_best_weights=True),
      TerminateOnNaN()
  ])

  tmp = model.get_weights()

  v_loss = hist.history['val_loss'][-1]

  sols.append(recursiveEncoder(tmp))


In [ ]:
def marca(msg):
  def _decorator(f):
    def _function_wrapper(x):
      print("inicio",msg)
      return f(x)
    return _function_wrapper
    print("final",msg)
  return _decorator

In [ ]:
@marca('evaluación')
def evaluar(soluciones):
  sols = []
  for i in soluciones:
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.set_weights(recursiveDecoder(i))
    hist = model.fit(X,
      Y,
      batch_size=96,
      epochs=2000,
      validation_split=.3,
      callbacks=[
      EarlyStopping(monitor="val_loss",restore_best_weights=True),
      TerminateOnNaN()
    ])
    sols.append([recursiveEncoder(model.get_weights()), hist.history['val_loss'][-1]])
  return sols

In [ ]:
import random

def cruzar(sol1,sol2):
  ind = random.sample(range(len(sol2)), random.randint(1,len(sol2)))
  for i in ind:
    d = np.random.randint(1, len(sol1[i]))
    v1 = sol1[i][:d]+sol2[i][d:]
    v2 = sol2[i][:d]+sol1[i][d:]

    sol1[i] = v1
    sol2[i] = v2
  return [sol1,sol2]


@marca("cruzar")
def cruzarSols(soluciones):
  sl = soluciones.copy()
  for i in soluciones:
    if random.random()<pc:
      for j in soluciones:
        if j!=i:
          sl.extend(cruzar(i.copy(),j.copy()))
  return sl


In [ ]:
def mutar(sol):
  if isinstance(sol,str):
    tmp = list(sol)
    for i in random.sample(range(len(sol)), random.randint(1,len(sol))):
      tmp[i] = complement[tmp[i]] if tmp[i]!='.' else '.'
    return "".join(tmp)

  ind = random.sample(range(len(sol)), random.randint(1,len(sol)))
  for i in ind:
    sol[i] = mutar(sol[i])
  return sol

@marca("mutación")
def mutarSols(soluciones):
  sl = soluciones.copy()
  for i in soluciones:
    if random.random() < pm:
      sl.append(mutar(i.copy()))
  return sl

In [ ]:
for i in range(epoch):
  print('Entrenando',i)
  sols = cruzarSols(sols)
  sols = mutarSols(sols)
  temp = evaluar(sols)
  temp = sorted(temp, key=lambda x: x[1])[:5]
  sols = [i[0] for i in temp]

In [ ]:
#Restaurando pesos
model = Sequential(layers)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.set_weights(recursiveDecoder(temp[0][0]))
print(temp[0][1])

In [ ]:
np.argmax(model.predict(X),axis=1)